In [43]:
%matplotlib inline
import pandas as pd
import subprocess
import itertools
import numpy as np

# Configuracion
command = "./tp {input} {output} {method}"

tests = [
    {'input': 'experimento1-1',
     'output': 'experimento1-1-outputs/{method}-{h}.txt',
     'method': [0, 1],
     'skip': 1,
     'h': [0.5, 0.8, 1.0, 1.25, 2.0, 2.5, 4.0, 5.0, 6.25, 10.0, 12.5]},
    {'input': 'experimento1-2',
     'output': 'experimento1-2-outputs/{method}-{h}.txt',
     'method': [0, 1],
     'skip': 1,
     'h': [0.5, 0.8, 1.0, 1.25, 2.0, 2.5, 4.0, 5.0, 6.25, 10.0, 12.5]},
    {'input': 'experimento1-3',
     'output': 'experimento1-3-outputs/{method}-{h}.txt',
     'method': [0, 1],
     'h': [0.5, 0.8, 1.0, 1.25, 2.0, 2.5, 4.0, 5.0, 6.25, 10.0, 12.5]},
    {'input': 'experimento2-1',
     'output': 'experimento2-1-outputs/{name}-{method}-{h}.txt',
     'method': [0, 1],
     'runs': 5,
     'skip': 1,
     'h': [0.5, 0.8, 1.0, 1.25, 2.0, 2.5, 4.0, 5.0, 6.25, 10.0, 12.5]},
    {'input': 'experimento3-1',
     'inputs': ['experimento3-1-0', 'experimento3-1-1', 'experimento3-1-2', 'experimento3-1-3', 'experimento3-1-4', 'experimento3-1-5', 'experimento3-1-6', 'experimento3-1-7', 'experimento3-1-8', 'experimento3-1-9', 'experimento3-1-10'],
     'output': 'experimento3-1-outputs/{name}-{method}-{h}.txt',
     'method': [2, 3],
     'skip': 1,
     'h': [1]}]

# Generacion de datos
for file in tests:
    name = file['input']
    
    dataset = []
    
    try:
        if file['skip']:
            print 'Salteando experimento', name
            continue
    except:
        pass
    
    print 'Comenzando experimento', name
    
    try:
        names = file['inputs']
    except:
        names = [name]
        
    for nameI in names:
        print 'Corriendo subexperimento', nameI
        
        with open(nameI + '.input', 'r') as handle:
            file['contents'] = handle.read()

            for (run_method, run_h) in itertools.product(file['method'], file['h']):
                output = {}
                output['name'] = nameI
                output['method'] = run_method
                output['h'] = run_h

                with open('/tmp/tmp.test', 'w+') as writer:
                    writer.write(file['contents'].format(h=run_h))

                outFile = file['output'].format(name=nameI, h=run_h, method=run_method)
                
                try:
                    runs = file['runs']
                except:
                    runs = 1
                
                for r in xrange(runs):
                    fmt = command.format(input='/tmp/tmp.test', output=outFile, method=run_method)
                    runtime = subprocess.check_output(fmt, shell=True)

                    for line in runtime.split('\n'):
                        if line:
                            line = line.split(':')
                            line[0] = line[0].strip().lower()
                            line[1] = float(line[1].strip())
                            output[line[0]] = line[1]

                    dataset.append(output)

    dataset = pd.DataFrame(dataset)
    dataset.to_csv(name + '.csv', index=False)
    print 'CSV guardado'
        
    print 'Experimento', name, 'finalizado'

Salteando experimento experimento1-1
Salteando experimento experimento1-2
Comenzando experimento experimento1-3
Corriendo subexperimento experimento1-3
CSV guardado
Experimento experimento1-3 finalizado
Salteando experimento experimento2-1
Salteando experimento experimento3-1


In [ ]:
# Experimento 1-1
# Muchas sanguijuelas pequeñas cerca del punto crítico, la idea es que a medida que aumentemos la granularidad
# van a empezar a aparecer en la discretización, haciendo que la temperatura del punto crítico vaya tornándose
# cada vez más alta.
import numpy.random as npr

n=50
width=100
height=100

minX = width/2 - 5
maxX = width/2 + 5
minY = height/2 - 5
maxY = height/2 + 5

posX = npr.uniform(minX, maxX, n)
posY = npr.uniform(minY, maxY, n)
radius = map(abs, npr.normal(0.5, max([maxX - minX, maxY - minY]), n))
temperatures = npr.exponential(100, n)

output = zip(posX, posY, radius, temperatures)

print width, height, "{h}", n
for (x, y, r, t) in output:
    print x, y, r, t

In [ ]:
# Experimento 1-2
# Sanguijuelas alejadas, de un radio comparativamente grande.
# Lo que debería suceder en este caso es que no se modificaría la temperatura del punto crítico a medida
# que incrementemos la granularidad.
import numpy.random as npr

n=50
width=100
height=100

posX = npr.uniform(0, width, n)
posY = npr.uniform(0, height, n)
radius = map(abs, npr.normal(2, max(width, height), n))
temperatures = npr.exponential(100, n)

output = zip(posX, posY, radius, temperatures)

print width, height, "{h}", n
for (x, y, r, t) in output:
    print x, y, r, t

In [41]:
# Experimento 1-3
# Tomar sanguijuelas pequeñas, alejadas del centro, con temperaturas muy altas.
# Si las temperaturas son lo suficientemente altas se deberían propagar hacia el centro a medida que aumente h
import numpy.random as npr

n=50
width=100
height=100

posX = npr.uniform(0, width, n)
posY = npr.uniform(0, height, n)
radius = map(abs, npr.normal(0.1, max(width, height)/10, n))
temperatures = npr.exponential(200, n)

output = zip(posX, posY, radius, temperatures)

print width, height, "{h}", n
for (x, y, r, t) in output:
    print x, y, r, t

100 100 {h} 50
6.61000302974 10.8696188011 2.38322636462 211.777989066
18.565912874 75.9346170811 5.30884644556 1.89463964613
16.09732459 34.9165604287 13.5191467846 32.9148072627
44.8809404334 81.4501350789 17.0069041862 94.5891221972
23.8985600926 45.6209612219 11.9461500331 46.2863577129
39.6707818705 39.738887453 4.81453630658 154.825547302
0.439712380716 77.1117877932 6.31326668045 326.744166859
88.3729457943 18.3326887444 4.13415882112 530.57071564
30.4966727188 66.3950439636 0.826439646801 283.667023615
65.9060969619 55.5284998512 18.9865846315 140.064917326
10.304160842 89.6708056931 5.31874186189 241.230264096
81.0616222024 75.8838245692 8.22180554582 278.644146373
69.2230969254 34.5909874512 19.1631655194 105.763057192
52.3311246663 95.7064033121 2.35830454692 122.974160355
19.6202647832 65.23946198 12.2201142651 12.2574177833
18.4758128662 60.5429049385 5.19445383744 70.9972694491
33.8794490404 7.74319604788 5.05407696404 349.68418338
26.4226108499 89.2164648141 12.279236885

In [ ]:
# Experimento 2-1
# Este experimento es solo para ver que pasa con las sanguis
import numpy.random as npr

n=50
width=100
height=100

posX = npr.uniform(0, 10, n)
posY = npr.uniform(0, 10, n)
radius = npr.uniform(0, 10, n)
temperatures = npr.exponential(300, n)

output = zip(posX, posY, radius, temperatures)

print width, height, "{h}", n
for (x, y, r, t) in output:
    print x, y, r, t

In [30]:
# Experimento 3-1
# Tomamos una grilla de sanguijuelas situadas en los puntos de la discretización (no variamos el h)
# y vamos aumentando la cantidad de sanguijuelas unitarias (es decir, las que tienen radio < h).

# IMPORTANTE: Si se cambia el h, cambiarlo en el primer casillero también
import numpy as np
import numpy.random as npr
import copy
import itertools

width=100
height=100
h = 1
percentages = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
outputs = ['experimento3-1-0', 'experimento3-1-1', 'experimento3-1-2', 'experimento3-1-3', 'experimento3-1-4', 'experimento3-1-5', 'experimento3-1-6', 'experimento3-1-7', 'experimento3-1-8', 'experimento3-1-9', 'experimento3-1-10']

posX = list(np.arange(0, width, h))
posY = list(np.arange(0, height, h))

# Aca comienza el codigo

n = 31
radiusLower = list(npr.uniform(0.2, h - 0.2, n - 1))
print radiusLower
radiusHigher = list(npr.uniform(h + 0.1, h + 0.4, n - 1))
temperatures = list(npr.exponential(300, n - 1))
positions = list(itertools.product(posX, posY))

for (fname, x) in zip(outputs, percentages):
    with open(fname + '.input', 'w+') as handle:        
        print >> handle, width, height, "{h}", n
        leftPositions = copy.deepcopy(positions)
        leftRL = copy.deepcopy(radiusLower)
        leftRH = copy.deepcopy(radiusHigher)
        leftT = copy.deepcopy(temperatures)

        noUnitarias = round(x*float(n - 1))
        unitarias = float(n - 1) - noUnitarias
        
        t = noUnitarias
        noUnitarias = unitarias
        unitarias = t
        print unitarias, noUnitarias

        i = 0
        while i < unitarias:
            # Generamos la cantidad de sanguijuelas unitarias que hagan falta
            (x, y) = leftPositions.pop()
            r = leftRL.pop()
            t = leftT.pop()

            print >> handle, x, y, r, t
            i += 1.0

        i = 0
        while i < noUnitarias:
            # El resto son sanguijuelas no unitarias
            (x, y) = leftPositions.pop()
            r = leftRH.pop()
            t = leftT.pop()

            print >> handle, x, y, r, t
            i += 1.0
        
        print >> handle, (width/2)/h, (height/2)/h, h - 0.4354, 1000.0

[0.73375020126714352, 0.77704991609135532, 0.3752746091405188, 0.45363763615870983, 0.4342259936849524, 0.48858273988913953, 0.63700273420250064, 0.58301357017597633, 0.61245516781487397, 0.61023191432081458, 0.53441229872397011, 0.64539097000900125, 0.29581252920873613, 0.32348586041241112, 0.54085727462825894, 0.43549095607970062, 0.75002780383054479, 0.77815749820053615, 0.37992023331517538, 0.69272307585152459, 0.73292886840060389, 0.44711178437322407, 0.38245500238115482, 0.79118393652752173, 0.78939862468618771, 0.55690802998260502, 0.57112566175195878, 0.62048134931345555, 0.29606053738156168, 0.20574538544666907]
0.0 30.0
3.0 27.0
6.0 24.0
9.0 21.0
12.0 18.0
15.0 15.0
18.0 12.0
21.0 9.0
24.0 6.0
27.0 3.0
30.0 0.0


In [32]:
# Selección de los h para la experimentación, queremos ver qué h sirven con el width y height que buscamos.
# Si hay más de 50 000 de dims, va a ser complicado que el experimento corra bien, asi que el h tiene que
# estar por debajo de eso para que la cosa ande.
width=100.0
height=100.0

lst = [x for x in np.arange(0.01, 100, 0.01) if round(width/x) == width/x and round(height/x) == height/x]
[(x, (width/x)*(height/x)) for x in lst]

[(0.01, 100000000.0),
 (0.02, 25000000.0),
 (0.040000000000000001, 6250000.0),
 (0.050000000000000003, 4000000.0),
 (0.080000000000000002, 1562500.0),
 (0.16, 390625.0),
 (0.20000000000000001, 250000.0),
 (0.25, 160000.0),
 (0.40000000000000002, 62500.0),
 (0.5, 40000.0),
 (0.80000000000000004, 15625.0),
 (1.0, 10000.0),
 (1.25, 6400.0),
 (2.0, 2500.0),
 (2.5, 1600.0),
 (4.0, 625.0),
 (5.0, 400.0),
 (6.25, 256.0),
 (10.0, 100.0),
 (12.5, 64.0),
 (50.0, 4.0)]

In [33]:
map(lambda (x, y): x, [(0.5, 40000.0), (0.80000000000000004, 15625.0), (1.0, 10000.0), (1.25, 6400.0), (2.0, 2500.0), (2.5, 1600.0), (4.0, 625.0), (5.0, 400.0), (6.25, 256.0), (10.0, 100.0), (12.5, 64.0), (50.0, 4.0)])

[0.5, 0.8, 1.0, 1.25, 2.0, 2.5, 4.0, 5.0, 6.25, 10.0, 12.5, 50.0]